In [1]:
import open3d as o3d
import numpy as np
import math
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [21]:
def write_pcd(file, out_file='pcd.txt', height=0):
    """ write one measurement per angle to out file (in euclidean format) """
    angles = []
    distances = []
    seen = set()

    with open(file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            angle, dist, quality = map(lambda x : float(x.split(':')[1]), line.split())
            angle = math.floor(angle)
            if angle not in seen:
                angles.append(math.radians(angle))
                distances.append(dist)
                seen.add(angle)

    angles = np.array(angles)
    distances = np.array(distances)
    
    points_x = np.cos(angles) * distances
    points_y = np.sin(angles) * distances

    # write in x y z format for open3d
    with open(out_file, 'w') as f:
        for x, y in zip(points_x, points_y):
            f.write(f'{x} {y} {height}\n')

##### Convert files to required xyz format

In [2]:
BASE_PATH_RAW_POINTS = "../data/out"
BASE_PATH_POINTS = "../data/out_uni"

##### Convert to required format

In [22]:
write_pcd(f'{BASE_PATH_RAW_POINTS}/1.txt', out_file=f'{BASE_PATH_POINTS}/1.txt')
write_pcd(f'{BASE_PATH_RAW_POINTS}/2.txt', out_file=f'{BASE_PATH_POINTS}/2.txt')
write_pcd(f'{BASE_PATH_RAW_POINTS}/3.txt', out_file=f'{BASE_PATH_POINTS}/3.txt')
write_pcd(f'{BASE_PATH_RAW_POINTS}/4.txt', out_file=f'{BASE_PATH_POINTS}/4.txt')


##### Load point clouds

In [3]:
pcd_1 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/1.txt", format='xyz')
pcd_2 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/2.txt", format='xyz')
pcd_3 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/3.txt", format='xyz')
pcd_4 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/4.txt", format='xyz')

### Scan matching
Since we have a 2D lidar, we have to take scans of the room at multiple heights to construct a model of the room. A 2D lidar gives a floor plan like output

In [4]:
# sample scan
o3d.visualization.draw_geometries([pcd_1])

We take measurements at different heights and stack them to get a 3D model. The scans taken at each height **will not be of the same orientation** due to slight movements. This is true in the case of a drone where vibrations and small translations could give a **translated and/or rotated scan**, when compared to the previous scan.

If we can identify some **keypoints/features/corresponding** points between 2 scans, we can match them by applying a rigid body transformation (a room is rigid).


In [6]:
# view subsequent scans
o3d.visualization.draw_geometries([pcd_1, pcd_3])

A simple heuristic will be the closest neighbour of a point. If we consider a point in a scan and find the closest point to it in the next scan, there is a high change that those **2 points have a correspondence** (here correspondence means that those points are of the same location in the room).

##### To align the point clouds, we need to minimize the distance between these corresponding pairs of points.

Once we find the distance between the point pairs, we estimate a rigid body transformation which will align these 2 scans. Solving a system of linear equations gives us a rotation and translation matrix.

Since we've used a heuristic, we don't know whether this is the optimal match for the scans. We use an error metric like root mean squared error to calculate the distance between the 2 scans, _after_ the transformation. **Our goal is to minimize this RMSE**, we repeat the above steps until the error(distance between scans) is low enough. 

Since the error metric calculates the point to point euclidean distance, the algorithm is called Point to Point ICP.

![](../images/ICP.png)

##### 1) We don't know the corresponding points, find them using the nearest neighbour heuristic.

##### 2) Once we have the corresponding points, find the transformation that minimizes the distance between the point clouds (Distance measure is Euclidean distance). Given corresponding points, we know that one point cloud is **translated and rotated** by some matrix T and R respectively, from the target point cloud.

<img src="../images/given.png"/>
<br />

##### 3) The error metric used is squared error, we have to solve this least squares problem to get the optimal transformation.
<img src="../images/problem.png" />
<br />

##### 4) We can move the both the point clouds to the origin, by subtracting the centroids of each point cloud
<img src="../images/com.png" />
<br />

##### 5) We can solve this problem in one shot by calculating the SVD of the given matrix
<img src="../images/svd.png" />
<br />

##### 6) The optimal matrices are 
<img src="../images/matrices.png" />

In [38]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source, source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])

In [7]:
pcd_1_copy = copy.deepcopy(pcd_1)
pcd_2_copy = copy.deepcopy(pcd_2)
pcd_3_copy = copy.deepcopy(pcd_3)
pcd_4_copy = copy.deepcopy(pcd_4)

In [8]:
# this is our base scan, we are matching all other scans to this
target = pcd_1_copy
# other point clouds
sources = [pcd_2_copy, pcd_3_copy, pcd_4_copy]

# initial transformation guess
trans_init = np.eye(4,4)

for source in sources:

    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, 
        target, 
        200, 
        trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000)
    )

    print(reg_p2p)
    print("Transformation is:")
    print(reg_p2p.transformation)

    source.transform(reg_p2p.transformation)


RegistrationResult with fitness=9.972222e-01, inlier_rmse=1.462995e+01, and correspondence_set size of 359
Access transformation to get result.
Transformation is:
[[ 9.99348867e-01  3.60810385e-02  0.00000000e+00  3.88657061e+01]
 [-3.60810385e-02  9.99348867e-01  0.00000000e+00  1.61643649e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.493350e+01, and correspondence_set size of 360
Access transformation to get result.
Transformation is:
[[  0.97493107   0.2225071    0.          13.56550903]
 [ -0.2225071    0.97493107   0.         -61.92267342]
 [  0.           0.           1.           0.        ]
 [  0.           0.           0.           1.        ]]
RegistrationResult with fitness=1.000000e+00, inlier_rmse=2.657564e+01, and correspondence_set size of 360
Access transformation to get result.
Transformation is:
[[  0.98285917  -0.18

In [9]:
def visualise(pcd_1, pcd_2, pcd_3, pcd_4, pcd_1_copy, pcd_2_copy, pcd_3_copy, pcd_4_copy):
    # Original point cloud
    pcd_1.translate((0,0,600))
    pcd_2.translate((0,0,1000))
    pcd_3.translate((0,0,1400))
    pcd_4.translate((0,0,1800))

    pcd_1.paint_uniform_color([0.8, 0, 0])
    pcd_2.paint_uniform_color([0, 0.8, 0])
    pcd_3.paint_uniform_color([0, 0, 0.8])
    pcd_4.paint_uniform_color([1, 0.706, 0])


    # matched point cloud
    pcd_1_copy.translate((6000,0,600))
    pcd_2_copy.translate((6000,0,1000))
    pcd_3_copy.translate((6000,0,1400))
    pcd_4_copy.translate((6000,0,1800))

    pcd_1_copy.paint_uniform_color([0.8, 0, 0])
    pcd_2_copy.paint_uniform_color([0, 0.8, 0])
    pcd_3_copy.paint_uniform_color([0, 0, 0.8])
    pcd_4_copy.paint_uniform_color([1, 0.706, 0])

    o3d.visualization.draw_geometries(
        [
            pcd_1_copy, pcd_2_copy, pcd_3_copy, pcd_4_copy,
            pcd_1, pcd_2, pcd_3, pcd_4
        ]
    )


In [10]:
visualise(pcd_1, pcd_2, pcd_3, pcd_4, pcd_1_copy, pcd_2_copy, pcd_3_copy, pcd_4_copy)

### Assumptions
- We assume that subsequent scans will have some corresponding points. 
- If the lidar frequency is slower than the movement of the drone, there wont be much overlap between subsequent scans.
- ICP works best when the movements are not too drastic

#### Drastic change in subsequent scans will fail to match

In [14]:
pcd_1_temp = o3d.io.read_point_cloud("../data/sme_uni/sm_points_3.txt", format='xyz')
pcd_2_temp = o3d.io.read_point_cloud("../data/sme_uni/sm_points_side.txt", format='xyz')

pcd_1_temp_c = copy.deepcopy(pcd_1_temp)
pcd_2_temp_c = copy.deepcopy(pcd_2_temp)

pcd_1_temp_c.paint_uniform_color([1, 0.706, 0])
pcd_1_temp.paint_uniform_color([1, 0.706, 0])
pcd_2_temp.paint_uniform_color([0, 0.651, 0.929])
pcd_2_temp_c.paint_uniform_color([0, 0.651, 0.929])

o3d.visualization.draw_geometries([pcd_1_temp, pcd_2_temp])

In [15]:
reg_p2p = o3d.pipelines.registration.registration_icp(
        pcd_2_temp_c, 
        pcd_1_temp_c, 
        300, 
        np.eye(4,4),
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000)
    )

pcd_2_temp_c.transform(reg_p2p.transformation)


pcd_1_temp.translate((0, 0, 0))
pcd_2_temp.translate((0, 0, 600))
pcd_1_temp_c.translate((6000, 0, 0))
pcd_2_temp_c.translate((6000, 0, 600))

o3d.visualization.draw_geometries([pcd_1_temp, pcd_2_temp, pcd_1_temp_c, pcd_2_temp_c])

---
---

#### Convert to mesh

In [44]:
# combine reigistered point clouds
pcd_1_pts = np.asarray(pcd_1_copy.points)
pcd_2_pts = np.asarray(pcd_2_copy.points)
pcd_3_pts = np.asarray(pcd_3_copy.points)
pcd_4_pts = np.asarray(pcd_4_copy.points)

final_points = np.vstack((pcd_1_pts, pcd_2_pts, pcd_3_pts, pcd_4_pts))

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(final_points)

# computer mesh using ball pivoting
# http://www.open3d.org/docs/latest/tutorial/Advanced/surface_reconstruction.html#Ball-pivoting

pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(100)
pcd.normals = o3d.utility.Vector3dVector(-np.asarray(pcd.normals))

distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 5 * avg_dist

# print(radius)

radii = [radius, radius * 1.5, radius * 2]
radii = [radius * 2, radius * 2.5]

bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd,
    o3d.utility.DoubleVector(radii)
)

bpa_mesh.remove_degenerate_triangles()
bpa_mesh.remove_duplicated_triangles()
bpa_mesh.remove_duplicated_vertices()
bpa_mesh.remove_non_manifold_edges()


TriangleMesh with 1440 points and 1594 triangles.

In [46]:
pcd.paint_uniform_color([0, 0.5, 0])
bpa_mesh.paint_uniform_color([0, 0, 0.5])
o3d.visualization.draw_geometries([pcd, bpa_mesh])
# o3d.visualization.draw_geometries([bpa_mesh])

In [40]:
def add_color_to_mesh(mesh):
    triangles = np.asarray(mesh.triangles).copy()
    vertices = np.asarray(mesh.vertices).copy()

    triangles_3 = np.zeros_like(triangles)
    vertices_3 = np.zeros((len(triangles) * 3, 3), dtype=vertices.dtype)
    vertex_colors = np.zeros((len(triangles) * 3, 3), dtype=np.float64)

    for index_triangle, t in enumerate(triangles):
        index_vertex = index_triangle * 3
        vertices_3[index_vertex] = vertices[t[0]]
        vertices_3[index_vertex + 1] = vertices[t[1]]
        vertices_3[index_vertex + 2] = vertices[t[2]]

        vertex_colors[index_vertex] = 0.5
        vertex_colors[index_vertex + 1] = 0
        vertex_colors[index_vertex + 2] = 0.8

        triangles_3[index_triangle] = np.arange(index_vertex, index_vertex + 3)

    mesh_return = copy.deepcopy(mesh)
    mesh_return.triangles = o3d.utility.Vector3iVector(triangles_3)
    mesh_return.vertices = o3d.utility.Vector3dVector(vertices_3)
    mesh_return.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)

    return mesh_return


colored_mesh = add_color_to_mesh(bpa_mesh)

# o3d.visualization.draw_geometries([pcd, colored_mesh])
o3d.visualization.draw_geometries([colored_mesh])

### Coherent point drift

In [11]:
pcd_1 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/1.txt", format='xyz')
pcd_2 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/2.txt", format='xyz')
pcd_3 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/3.txt", format='xyz')
pcd_4 = o3d.io.read_point_cloud(f"{BASE_PATH_POINTS}/4.txt", format='xyz')

In [12]:
import copy
from probreg import cpd

pcd_1_copy = copy.deepcopy(pcd_1)
pcd_2_copy = copy.deepcopy(pcd_2)
pcd_3_copy = copy.deepcopy(pcd_3)
pcd_4_copy = copy.deepcopy(pcd_4)

# this is our base scan, we are matching all other scans to this
target = pcd_1_copy
# other point clouds
sources = [pcd_2_copy, pcd_3_copy, pcd_4_copy]

for source in sources:

# compute cpd registration
    tf_param, _, _ = cpd.registration_cpd(source, target)
    source.points = tf_param.transform(source.points)

    # draw result
    source.paint_uniform_color([1, 0, 0])
    target.paint_uniform_color([0, 1, 0])
    o3d.visualization.draw_geometries([source, target])

ModuleNotFoundError: No module named 'probreg'

In [ ]:
visualise(pcd_1, pcd_2, pcd_3, pcd_4, pcd_1_copy, pcd_2_copy, pcd_3_copy, pcd_4_copy)